In [ ]:
## Update the Config.yaml

data_validation:
    root_dir : artifacts/data_validation
    unzip_data_dir : artifacts/data_ingestion/winequality-red.csv
    STATUS_FILE: artifacts/data_validation/status.txt


In [ ]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')

In [ ]:
# Update the Schema
COLUMNS:
    fixed acidity: float64
    volatile acidity: float64
    citric acid: float64
    residual sugar: float64
    chlorides: float64
    free sulfur dioxide: float64
    total sulfur dioxide: float64
    density: float64
    pH: float64
    sulphates: float64
    alcohol: float64
    quality: int64

TARGET_COLUMN:
    name: quality



In [ ]:
data.isnul().sum()
data.shape

In [ ]:
# Update the config.Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    local_data_file: Path
    unzip_dir: Path
    all_schema: dict

In [ ]:
# Update the Configuration Manager --> Configuration.py

from src.end_to_end_ML_project_1.constants import *
from src.end_to_end_ML_project_1.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    ## Get Data Validation Configuration
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config


In [ ]:
## update the Components

# Perform the Data Validation by comparing the data to the schema
import os
from src.end_to_end_ML_project_1 import logger
from src.end_to_end_ML_project_1.entity.config_entity import DataValidationConfig

class DataValidation:
    def __init__(self,config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f'Validation Status: {validation_status}')
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation Status: {validation_status}')
            return validation_status
        except Exception as e:
            raise e




In [ ]:
## update the pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

